In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinyllama-final/adapter_model.safetensors
/kaggle/input/tinyllama-final/trainer_state.json
/kaggle/input/tinyllama-final/training_args.bin
/kaggle/input/tinyllama-final/adapter_config.json
/kaggle/input/tinyllama-final/README.md
/kaggle/input/tinyllama-final/tokenizer.json
/kaggle/input/tinyllama-final/tokenizer_config.json
/kaggle/input/tinyllama-final/scaler.pt
/kaggle/input/tinyllama-final/scheduler.pt
/kaggle/input/tinyllama-final/special_tokens_map.json
/kaggle/input/tinyllama-final/optimizer.pt
/kaggle/input/tinyllama-final/rng_state.pth
/kaggle/input/tinyllama-final/tokenizer.model


In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
# ✅ STEP 1: Load your fine-tuned TinyLLaMA model and tokenizer

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/kaggle/input/tinyllama-final"  # replace with your actual model folder path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")
model.eval()

print("✅ Model loaded and ready.")


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

2025-06-21 06:24:35.727373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750487075.948328      59 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750487076.008957      59 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model loaded and ready.


In [5]:
# ✅ STEP 2: Define a function to generate multiple responses for each prompt

def generate_multiple_responses(prompt, num_variants=3):
    variants = []
    for i in range(num_variants):
        inputs = tokenizer(f"<|user|>: {prompt} <|assistant|>", return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=500,
                temperature=0.7 + i * 0.2,  # slight variation
                top_p=0.85 - i * 0.1,
                do_sample=True,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        variants.append(text)
    return variants

In [6]:
# ✅ STEP 3: Run this on some test prompts and store results

medical_prompts = [
    "What are the symptoms of dengue fever?",
    "I have sore throat and fever for 2 days. What could be the cause?",
    "How do I manage high blood pressure at home?",
    "Can you explain asthma simply?",
    "What causes chest pain while breathing deeply?"
]

response_dataset = []

for prompt in medical_prompts:
    print(f"\n🔍 Generating for: {prompt}")
    variants = generate_multiple_responses(prompt)
    for i, v in enumerate(variants):
        print(f"\n🔹 Variant {i+1}:\n{v}\n")
    response_dataset.append({"prompt": prompt, "responses": variants})


🔍 Generating for: What are the symptoms of dengue fever?

🔹 Variant 1:
<|user|>: What are the symptoms of dengue fever? <|assistant|>

Dengue fever is a viral disease caused by the dengue virus. Here are some common symptoms:

1. Fever: Most people with dengue fever experience high fever (over 102°F) for several days.

2. Rash: Dengue fever can cause a rash, which usually appears as small red spots on the skin. The rash usually appears within 2-3 days after the onset of symptoms.

3. Headache: Some people may experience a headache as a symptom of dengue fever.

4. Muscle pain: Some people may experience muscle pain, especially in the lower back, hips, and thighs.

5. Joint pain: Some people may experience joint pain, especially in the wrists, ankles, and knees.

6. Nausea and vomiting: Some people may experience nausea and vomiting as a symptom of dengue fever.

7. Abdominal pain: Some people may experience abdominal pain, especially in the abdomen or lower back.

8. Fatigue: Some peo

In [7]:
# ✅ STEP 4: Save to JSON or CSV for ranking (manually or using notebook UI)
import json

with open("rag_responses_for_rhlf.json", "w") as f:
    json.dump(response_dataset, f, indent=2)

print("\n✅ Saved all generated responses to 'rag_responses_for_rhlf.json' — Ready for ranking!")


✅ Saved all generated responses to 'rag_responses_for_rhlf.json' — Ready for ranking!


# Ranking responses

In [8]:
import json

# Load your response dataset
with open("/kaggle/working/rag_responses_for_rhlf.json", "r") as f:
    data = json.load(f)

ranked_data = []

for item in data:
    print(f"\n📌 Prompt: {item['prompt']}")
    responses = item['responses']

    # Display responses with index
    for i, r in enumerate(responses):
        print(f"\n🔹 Variant {i}: \n{r.strip()}\n")

    # Ask for rankings
    ranks = input(f"👉 Rank the responses (comma-separated 0-2): e.g., 2,1,0\n")
    rank_list = [int(r.strip()) for r in ranks.split(",")]

    for i in range(len(responses)):
        ranked_data.append({
            "prompt": item["prompt"],
            "response": responses[i],
            "rank": rank_list[i]
        })

# Save ranked data for reward model training
with open("ranked_rhlf_data.json", "w") as f:
    json.dump(ranked_data, f, indent=2)

print("\n✅ Done! Saved ranked responses to 'ranked_rhlf_data.json'")



📌 Prompt: What are the symptoms of dengue fever?

🔹 Variant 0: 
<|user|>: What are the symptoms of dengue fever? <|assistant|>

Dengue fever is a viral disease caused by the dengue virus. Here are some common symptoms:

1. Fever: Most people with dengue fever experience high fever (over 102°F) for several days.

2. Rash: Dengue fever can cause a rash, which usually appears as small red spots on the skin. The rash usually appears within 2-3 days after the onset of symptoms.

3. Headache: Some people may experience a headache as a symptom of dengue fever.

4. Muscle pain: Some people may experience muscle pain, especially in the lower back, hips, and thighs.

5. Joint pain: Some people may experience joint pain, especially in the wrists, ankles, and knees.

6. Nausea and vomiting: Some people may experience nausea and vomiting as a symptom of dengue fever.

7. Abdominal pain: Some people may experience abdominal pain, especially in the abdomen or lower back.

8. Fatigue: Some people may

👉 Rank the responses (comma-separated 0-2): e.g., 2,1,0
 0,1,2



📌 Prompt: I have sore throat and fever for 2 days. What could be the cause?

🔹 Variant 0: 
<|user|>: I have sore throat and fever for 2 days. What could be the cause? <|assistant|>

Sore throat and fever are both common symptoms of upper respiratory infections (URIs), also known as respiratory tract infections (RTIs). The most common cause of URIs is viral infection, such as influenza (flu), adenovirus, or rhinovirus. These infections cause inflammation of the nose, throat, and lungs and can lead to a sore throat, fever, and cough. In severe cases, URIs can lead to pneumonia, which is an infection of the lungs. It is essential to seek medical attention if you experience these symptoms for more than 24 hours or if they worsen or persist. A healthcare professional will conduct a thorough medical evaluation to identify the underlying cause and provide appropriate treatment.


🔹 Variant 1: 
<|user|>: I have sore throat and fever for 2 days. What could be the cause? <|assistant|>

Answer: 

👉 Rank the responses (comma-separated 0-2): e.g., 2,1,0
 2,1,0



📌 Prompt: How do I manage high blood pressure at home?

🔹 Variant 0: 
<|user|>: How do I manage high blood pressure at home? <|assistant|>
Sure! Managing high blood pressure at home can be done through a few simple steps. Here are some tips:

1. Regular exercise: Regular exercise can help lower blood pressure by increasing blood flow and reducing stress. Try to get at least 150 minutes of moderate-intensity exercise per week, such as brisk walking, cycling, or swimming.

2. Moderate salt intake: Aim for a sodium intake of around 2,300 mg per day. This is the equivalent of eating 1.5 teaspoons of salt. Too much salt can increase blood pressure, so aim to reduce it.

3. Avoid processed foods: Processed foods are often high in salt and can contribute to high blood pressure. Limit your intake of processed foods and opt for whole foods instead.

4. Eat a balanced diet: A balanced diet that includes plenty of fruits, vegetables, whole grains, and lean protein can help lower blood pressure.


...


📌 Prompt: Can you explain asthma simply?

🔹 Variant 0: 
<|user|>: Can you explain asthma simply? <|assistant|> Sure! Asthma is a chronic lung disease that causes inflammation and tightening of the bronchial tubes in the lungs, which can lead to difficulty breathing and wheezing. It is a common respiratory condition that affects millions of people worldwide. Asthma symptoms can include wheezing, coughing, shortness of breath, and chest tightness. The condition can be triggered by factors such as allergens, irritants, or infections, and can worsen with exercise or stress. Asthma can be managed with medication, exercise, and lifestyle changes such as avoiding triggers and avoiding smoking. Overall, asthma is a complex condition that requires careful diagnosis and treatment.


🔹 Variant 1: 
<|user|>: Can you explain asthma simply? <|assistant|> Sure! Asthma is a chronic condition that affects the airways in the lungs. The airways are the tubes that carry air in and out of the lungs. When 

👉 Rank the responses (comma-separated 0-2): e.g., 2,1,0
 0,1,2



📌 Prompt: What causes chest pain while breathing deeply?

🔹 Variant 0: 
<|user|>: What causes chest pain while breathing deeply? <|assistant|>

Chest pain while breathing deeply is a symptom of heart disease. It is caused by a narrowing of the arteries that supply blood to the heart, which leads to a decrease in blood flow to the heart. This causes a decrease in the amount of oxygen-rich blood that reaches the heart, which can lead to feelings of heaviness or discomfort in the chest. It is common for people with heart disease to experience chest pain while breathing deeply, especially during physical activity or when exercising. Chest pain can also occur during rest or sleep, especially if the person is lying down or sitting for long periods. Over time, heart disease can cause permanent damage to the heart muscle, which can lead to heart failure. It is essential to seek medical attention if you experience chest pain while breathing deeply, especially if it occurs often or worsens with

👉 Rank the responses (comma-separated 0-2): e.g., 2,1,0
 0,1,2



✅ Done! Saved ranked responses to 'ranked_rhlf_data.json'


# Train the Reward Model

In [10]:
from transformers import AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
import json

# PEFT / LoRA
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForSequenceClassification

# Load your ranked data
with open("ranked_rhlf_data.json", "r") as f:
    data = json.load(f)

# Prepare dataset
def preprocess(example):
    return {
        "text": f"<|user|>: {example['prompt']} <|assistant|>: {example['response']}",
        "label": int(example["rank"])
    }

formatted_data = [preprocess(item) for item in data]
dataset = Dataset.from_list(formatted_data)

# Load tokenizer
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(base_model)

# ✅ Fix: Define padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=384
    )
    tokens["labels"] = example["label"]
    return tokens

tokenized = dataset.map(tokenize, batched=False)

# Load classification model
model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=3  # rank labels: 0, 1, 2
)

# ✅ Fix: Set padding token ID in model config
model.config.pad_token_id = tokenizer.pad_token_id

# Apply LoRA config to classification model
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config).to("cuda")

# Training arguments
training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=2,             # Reduced for memory
    gradient_accumulation_steps=4,             # Effective batch size = 8
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,                                 # Mixed precision for GPU efficiency
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer
)

# Train
trainer.train()

# Save LoRA adapter
trainer.save_model("./reward_model")
print("✅ LoRA-based reward model saved at './reward_model'")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_59/2773228962.py:78: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ LoRA-based reward model saved at './reward_model'


In [11]:
import shutil

# Zip the folder
shutil.make_archive("reward_model", 'zip', "/kaggle/working/reward_model")


'/kaggle/working/reward_model.zip'